In [ ]:
import numpy as np # for arrays, Series, List, Sets
import pandas as pd # for dataframes

#- 9 meses: 26483
#- 9 meses without package: 26006 
#- 9 meses without cobranding: 23646 
#- todo junto: 23191

In [ ]:
data = pd.read_csv('data.csv', sep='|', skipfooter=1, engine='python')

In [ ]:
data.shape

In [ ]:
data['client_id'].nunique()

In [ ]:
sorted(data['Month'].unique())

In [ ]:
client_month_counts = data.groupby('client_id')['Month'].nunique()
print(client_month_counts)

In [ ]:
clients_with_9_months = client_month_counts[client_month_counts == 9].index
print(clients_with_9_months.shape)

In [ ]:
data2 = data[data['client_id'].isin(clients_with_9_months)]

data2['client_id'].nunique()
# clientes con 9 meses

In [ ]:
data2.head()

In [ ]:
data2.shape

In [ ]:
data2['CreditCard_CoBranding'].value_counts().reset_index()

In [ ]:
# 6 meses training, 1 mes lead, 2 meses prediction
mask = (data2['Month'] == '2019-01-01') & (data2['CreditCard_CoBranding'] != 'No')

clients_to_remove = data2.loc[mask, 'client_id'].unique()

data3 = data2[~data2['client_id'].isin(clients_to_remove)]

data3['client_id'].nunique()
# clientes sin cobranding en el mes 6

In [ ]:
data3.shape

In [ ]:
mask2 = (data3['Month'] == '2019-01-01') & (data3['Package_Active'] != 'No')

clients_to_remove2 = data3.loc[mask2, 'client_id'].unique()

data4 = data3[~data3['client_id'].isin(clients_to_remove2)]

data4['client_id'].nunique()
# clientes sin paquete ni cobranding en el mes 6

In [ ]:
data4['Region'].value_counts(dropna=False).reset_index()


In [ ]:
data4.head()

# Algunos Identity Features:
# client_id, Month, Sex

# Algunos Transform Featues:
# Client_Age_grp, First_product_dt, Last_product_dt

In [ ]:
# se ven muchos clientes con region y creditProduct solo en los meses 19-3 y 19-4

data5 = data4.copy()

data_only_one = data5[(data5['client_id'] == 5856970)]
data_only_one[['client_id', 'Region', 'CreditCard_Product', 'Month']]

In [ ]:
# se encuentran 6 clientes que no tienen Region para ese mes

data_region = data5[(data5['Month'] == '2019-04-01')]
data_region[['Region', 'Month']].value_counts(dropna=False).reset_index()

In [ ]:
result_clients = data5[(data5['Month'] == '2019-04-01') & (data5['Region'].isna())]['client_id']
result_clients
# clientes sin regiones registradas en el mes 2019-04

In [ ]:
# se transfieren las 2 regiones repetidas de cada cliente, a todo el resto de sus meses

client_region_map = data5[data5['Month'] == '2019-04-01'].set_index('client_id')['Region'].to_dict()

data5['Region'] = data5['client_id'].map(client_region_map)


data_only_one = data5[(data5['client_id'] == 5856970)]
data_only_one[['client_id', 'Region', 'CreditCard_Product', 'Month']]

In [ ]:
# se inserta Desconocido en los 6 clientes con region desconocida

data5.loc[data5['client_id'].isin(result_clients), 'Region'] = 'Desconocido'

data_region2 = data5[(data5['Month'] == '2019-04-01')]
data_region2[['Region', 'Month']].value_counts(dropna=False).reset_index()

In [ ]:
data5['Region'].value_counts(dropna=False).reset_index()

# pasa de 162349 NaN hasta 54 desconocidos

In [ ]:
data5[['CreditCard_Product']].value_counts(dropna=False).reset_index()

In [ ]:
data_credit = data5[(data5['Month'] == '2019-04-01')]
data_credit[['CreditCard_Product', 'Month']].value_counts(dropna=False).reset_index()

In [ ]:
result_clients2 = data5[(data5['Month'] == '2019-04-01') & (data5['CreditCard_Product'].isna())]['client_id']
result_clients2
# clientes sin credit_product registradas en el mes 2019-04

In [ ]:
# se transfieren las 2 CreditCard_Product repetidos de cada cliente, a todo el resto de sus meses

data6 = data5.copy()

client_credit_map = data6[data6['Month'] == '2019-04-01'].set_index('client_id')['CreditCard_Product'].to_dict()

data6['CreditCard_Product'] = data6['client_id'].map(client_credit_map)


data_only_one2 = data6[(data6['client_id'] == 5856970)]
data_only_one2[['client_id', 'Region', 'CreditCard_Product', 'Month']]

In [ ]:
# se inserta Desconocido en los 8102 clientes con CreditCard_Product desconocido

data6.loc[data6['client_id'].isin(result_clients2), 'CreditCard_Product'] = 'Desconocido'

data_credit2 = data6[(data6['Month'] == '2019-04-01')]
data_credit2[['CreditCard_Product', 'Month']].value_counts(dropna=False).reset_index()

In [ ]:
data6[['CreditCard_Product']].value_counts(dropna=False).reset_index()

# pasa desde 178520 NaN, hasta 72918 desconocidos